In [35]:
import csv
import numpy
from ipyleaflet import Map, Marker, Circle

In [4]:
data =[]
with open('US_Accidents_May19.csv', mode='r') as file:
    csv_reader = csv.reader(file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        data_sample = {}
        if line_count==0:
            keys = row
        else:
            for i in range(len(keys)):
                data_sample[keys[i]] = row[i]
            data.append(data_sample)
        line_count += 1

In [20]:
# Filter function
def predicate(sample):
    if sample['State']==state and sample['City']==city and int(sample['Severity'])==severity:
        return True
    else:
        return False

# Filtering
def filter_input(data): return list(filter(predicate, data))

In [33]:
# Input Param
state = 'FL'
city = 'Jacksonville'
severity = 2

# Filter data by input param
output = filter_input(data)

# Print-out to check
print(len(output))

# Get coordinates from filter output (lat, long)
coordinates = []
lat_center, lng_center = 0, 0
for sample in output:
    coordinates.append((float(sample['Start_Lat']), float(sample['Start_Lng'])))
    lat_center += float(sample['Start_Lat'])
    lng_center += float(sample['Start_Lng'])
lat_center /= len(output)
lng_center /= len(output)

3598


In [ ]:
# Generate map
center = (lat_center, lng_center) # geological center of US
m = Map(center=center, zoom=8)
for i in range(len(coordinates)):
    circle = Circle()
    circle.location = coordinates[i]
    circle.radius = 500
    circle.color = "red"
    circle.fill_color = "red"
    m.add_layer(circle)
m